# Trouver les meilleurs clients

In [10]:
import pandas as pd
import duckdb

In [11]:
clients = ["Oussama", "Julie", "Chris", "Tom", "Jean-Nicolas", "Aline", "Ben", "Toufik", "Sylvie", "David"]
ventes = [110, 49, 65, 23, 24, 3.99, 29, 48.77, 44, 10, 60, 12, 62, 19, 75] * 2 

ventes = pd.DataFrame(ventes)
ventes.columns = ["montant"]
ventes["client"] = clients * 3
ventes.head(6)

,montant,client
0,110.00,Oussama
1,49.00,Julie
2,65.00,Chris
3,23.00,Tom
4,24.00,Jean-Nicolas
5,3.99,Aline


## Exercice, calculer le panier moyen pour chaque client

C'est à dire, faire simplement la moyenne des achats

In [7]:
%load solutions/5panier_moyen_client_python.py

client
Aline           57.996667
Ben             30.000000
Chris           58.590000
David           36.333333
Jean-Nicolas    36.333333
Julie           30.000000
Oussama         57.996667
Sylvie          28.666667
Tom             28.666667
Toufik          58.590000
Name: montant, dtype: float64

In [4]:
%load solutions/6panier_moyen_client_sql.py

┌──────────────┬────────────────────┐
│    client    │    total_ventes    │
│   varchar    │       double       │
├──────────────┼────────────────────┤
│ Oussama      │  57.99666666666667 │
│ Julie        │               30.0 │
│ Chris        │              58.59 │
│ Tom          │ 28.666666666666668 │
│ Jean-Nicolas │ 36.333333333333336 │
│ Aline        │  57.99666666666667 │
│ Ben          │               30.0 │
│ Toufik       │              58.59 │
│ Sylvie       │ 28.666666666666668 │
│ David        │ 36.333333333333336 │
├──────────────┴────────────────────┤
│ 10 rows                 2 columns │
└───────────────────────────────────┘

--- 

Ensuite, votre PO vous demande de sortir la liste des clients qui ont des achats supérieurs à la moyenne 

## Exercice: je triche avec Python, mais si vous étiez en interview SQL ?!

In [12]:
mean = (
    ventes["montant"].mean()
)
mean

42.31733333333333

In [113]:
query = f"""
SELECT client, 
MEAN(montant) as mean_sales
FROM ventes
GROUP BY CLIENT
HAVING mean_sales > {mean}
"""

duckdb.sql(query)

┌─────────┬───────────────────┐
│ client  │    total_sales    │
│ varchar │      double       │
├─────────┼───────────────────┤
│ Oussama │ 57.99666666666667 │
│ Chris   │             58.59 │
│ Aline   │ 57.99666666666667 │
│ Toufik  │             58.59 │
└─────────┴───────────────────┘

In [114]:
(
    ventes
    .groupby("client")
    ["montant"].mean()
    .reset_index()
    .query(f"montant > {mean}")
)

,client,montant
0,Aline,57.996667
2,Chris,58.590000
6,Oussama,57.996667
9,Toufik,58.590000


Faites la même chose en SQL sans utiliser une F-string Python:

In [17]:
# Avec une subquery
%load solutions/7mean_sales_python.py

┌─────────┬───────────────────┐
│ client  │    mean_sales     │
│ varchar │      double       │
├─────────┼───────────────────┤
│ Oussama │ 57.99666666666667 │
│ Chris   │             58.59 │
│ Aline   │ 57.99666666666667 │
│ Toufik  │             58.59 │
└─────────┴───────────────────┘

In [19]:
# Avec une CTE
%load solutions/8mean_sales_sql.py


┌─────────┬───────────────────┐
│ client  │    total_sales    │
│ varchar │      double       │
├─────────┼───────────────────┤
│ Oussama │ 57.99666666666667 │
│ Chris   │             58.59 │
│ Aline   │ 57.99666666666667 │
│ Toufik  │             58.59 │
└─────────┴───────────────────┘

## Exercice: la moyenne de la somme des ventes

Voici les ventes totales pour chaque client:

In [125]:
ventes.groupby("client")["montant"].sum()

client
Aline           173.99
Ben              90.00
Chris           175.77
David           109.00
Jean-Nicolas    109.00
Julie            90.00
Oussama         173.99
Sylvie           86.00
Tom              86.00
Toufik          175.77
Name: montant, dtype: float64

La moyenne de ces ventes totales est différente de celle de tout à l'heure:

In [126]:
ventes.groupby("client")["montant"].sum().mean()

126.952

Utilisez une CTE pour obtenir les clients qui ont un total d'achat supérieur à la moyenne des totaux d'achats des autres clients:

<img src="images/pirates-of-the-caribbean-captain-jack-sparrow.gif" />

(On veut les clients qui ont dépensé plus que 126.95€ :p)

Etapes:
- Faire une query pour obtenir les ventes totales par client
- La stocker dans une subquery
- A partir de cette subquery, faire un query pour obtenir la moyenne de ces ventes totales
- La stocker dans une 2nde subquery
- A partir de cette deuxième subquery, récupérer les clients et leur somme totales dépensées, et filtrer sur les clients dont la moyenne est supérieure à celle calculée dans la 2e subquery

In [26]:
query = """
WITH TOTAL_SALES_PER_CLIENT AS (
    SELECT SUM(montant) AS total_sales
    FROM ventes
    GROUP BY client
),

MEAN_TOTAL_SALES AS (
    SELECT MEAN(total_sales)
    FROM TOTAL_SALES_PER_CLIENT
)

SELECT client, 
SUM(montant) as total_sales
FROM ventes
GROUP BY CLIENT
HAVING total_sales > 
(SELECT * FROM MEAN_TOTAL_SALES)
"""

duckdb.sql(query)

┌─────────┬─────────────┐
│ client  │ total_sales │
│ varchar │   double    │
├─────────┼─────────────┤
│ Oussama │      173.99 │
│ Chris   │      175.77 │
│ Aline   │      173.99 │
│ Toufik  │      175.77 │
└─────────┴─────────────┘

# Les réunions, 2

Participants aux réunions:

In [32]:
person_names = ["Benjamin", "Florian", "Tarik", "Bob", "Sirine", "Alice"]

Création de la donnée sur les meetings:

In [33]:
import random

meetings_data = []
for meeting_id in range(150):
    persons_in_meet = random.sample(person_names, random.randint(1,5))
    for person_name in persons_in_meet:
        meetings_data.append((meeting_id, person_name))


meetings_df = pd.DataFrame(meetings_data, columns=["meeting_id", "person_name"])

Durées des meetings:

In [34]:
meeting_durations = []
for meeting_id in meetings_df["meeting_id"].unique():
    duration = random.randint(10, 45)  # You can adjust the range as needed
    meeting_durations.append((meeting_id, duration))

durations_df = pd.DataFrame(meeting_durations, columns=["meeting_id", "duration_minutes"])

Tweak de la donnée pour rendre l'exercice plus fun ;)

In [35]:
average_duration = durations_df["duration_minutes"].mean()
meetings_with_flo = meetings_df[meetings_df["person_name"] == "Florian"]["meeting_id"].unique()
# meetings_with_ben = meetings_df[meetings_df["person_name"] == "Benjamin"]["meeting_id"].unique()
# s=set(meetings_with_ben) & set(meetings_with_flo)
for _, row in durations_df.iterrows():
    if row["meeting_id"] in meetings_with_flo:
        row["duration_minutes"] += random.randint(50, 65)  # Add extra minutes to meet the condition

Total:

In [36]:
merged_df = meetings_df.merge(durations_df, on="meeting_id")
merged_df

,meeting_id,person_name,duration_minutes
0,0,Florian,88
1,0,Alice,88
2,1,Bob,74
3,1,Florian,74
4,1,Benjamin,74
...,...,...,...
446,148,Tarik,22
447,149,Florian,74
448,149,Alice,74
449,149,Bob,74


## Exercice 1: refaites le self join 

- créer une table avec toutes les combinaisons de personnes ayant assisté au même meeting
- ne garder que les records qui me concernent (Benjamin)
- enlever les records où je suis en réunion "avec moi-même"

In [38]:
# %load solutions/9self_join_review.py
query = """
SELECT meeting_id, 
rdf.person_name as colleague,
ldf.duration_minutes
FROM merged_df ldf
INNER JOIN merged_df rdf
USING (meeting_id)
WHERE ldf.person_name == 'Benjamin'
AND rdf.person_name != 'Benjamin'
"""

duckdb.sql(query)


┌────────────┬───────────┬──────────────────┐
│ meeting_id │ colleague │ duration_minutes │
│   int64    │  varchar  │      int64       │
├────────────┼───────────┼──────────────────┤
│          1 │ Alice     │               74 │
│          4 │ Sirine    │               37 │
│          6 │ Alice     │               94 │
│          7 │ Bob       │               85 │
│          8 │ Florian   │               69 │
│         12 │ Florian   │               88 │
│         13 │ Sirine    │               86 │
│         15 │ Florian   │              100 │
│         16 │ Tarik     │               24 │
│         17 │ Florian   │               77 │
│          · │   ·       │                · │
│          · │   ·       │                · │
│          · │   ·       │                · │
│        142 │ Tarik     │              110 │
│        146 │ Bob       │               35 │
│        148 │ Alice     │               22 │
│         44 │ Sirine    │               93 │
│         50 │ Florian   │        

## Exercice 2: calculez la durée moyenne des meetings avec chaque collegue 

Ensuite, faire un group by pour savoir la durée moyenne de mes meetings avec chaque personne
- [optionnel]: ne garder que les résultats pour lesquels la moyenne est > à 1h

In [40]:
# %load solutions/10reunions_solution.py
query = """ 
WITH meetings_benjamin AS (
    SELECT meeting_id, 
    rdf.person_name as colleague,
    ldf.duration_minutes
    FROM merged_df ldf
    INNER JOIN merged_df rdf
    USING (meeting_id)
    WHERE ldf.person_name == 'Benjamin'
    AND rdf.person_name != 'Benjamin'
)

SELECT colleague,
MEAN(duration_minutes) as avg_meeting_duration
FROM meetings_benjamin
GROUP BY colleague
"""

duckdb.sql(query)


┌───────────┬──────────────────────┐
│ colleague │ avg_meeting_duration │
│  varchar  │        double        │
├───────────┼──────────────────────┤
│ Alice     │   61.609756097560975 │
│ Sirine    │   60.470588235294116 │
│ Bob       │    66.23255813953489 │
│ Florian   │    85.69047619047619 │
│ Tarik     │                 69.0 │
└───────────┴──────────────────────┘